In [1]:
import sqlite3
import pandas as pd

In [3]:
df = pd.read_csv('../metadata/probe-coordinates.csv')

In [4]:
df

,probe_id,long,lat
0,10003,4.9375,52.3685
1,10006,0.1575,52.2205
2,10007,-122.3325,38.3305
3,10019,5.2875,43.5315
4,10023,13.3785,52.4985
...,...,...,...
5224,35733,49.5785,37.2785
5225,35734,36.2385,30.5895
5226,35735,-58.6715,-62.2405
5227,35742,31.0095,-29.8015


In [5]:
with sqlite3.connect('../data/pub-res.db') as conn:
    df_1 = pd.read_sql('select distinct prb_id, country_code, continent_code from do53_5pct', con=conn)
    df_2 = pd.read_sql('select distinct prb_id, country_code, continent_code from dot_5pct', con=conn)
    
with sqlite3.connect('../data/loc-res.db') as conn:
    df_3 = pd.read_sql('select distinct prb_id, country_code, continent_code from do53_5pct', con=conn)
    df_4 = pd.read_sql('select distinct prb_id, country_code, continent_code from dot_5pct', con=conn)

In [6]:
local_dot = df_4['prb_id'].unique()

In [7]:
local_dot

array([10342, 14377, 16233, 17525, 17609, 18275, 20324, 22890, 23214,
       29989, 30899, 32271, 32430])

In [8]:
df_new = pd.concat([df_1, df_2, df_3, df_4])

In [9]:
df_new

,prb_id,country_code,continent_code
0,10006,GB,EU
1,10007,US,NA
2,10080,JP,AS
3,10087,PL,EU
4,10092,BG,EU
...,...,...,...
8,23214,SE,EU
9,29989,NL,EU
10,30899,DE,EU
11,32271,RO,EU


In [10]:
df_new = df_new.drop_duplicates().rename(columns={'prb_id' : 'probe_id'})

In [11]:
df_new

,probe_id,country_code,continent_code
0,10006,GB,EU
1,10007,US,NA
2,10080,JP,AS
3,10087,PL,EU
4,10092,BG,EU
...,...,...,...
1491,24858,GB,EU
1681,27330,DE,EU
1843,28907,DE,EU
1846,28932,DE,EU


In [12]:
df_merged = df.merge(df_new, on='probe_id')

In [13]:
df_merged

,probe_id,long,lat,country_code,continent_code
0,10006,0.1575,52.2205,GB,EU
1,10007,-122.3325,38.3305,US,NA
2,10080,139.4315,35.9175,JP,AS
3,10087,20.6315,50.8805,PL,EU
4,10092,23.4015,42.6885,BG,EU
...,...,...,...,...,...
3157,35701,-95.3825,29.7475,US,NA
3158,35719,5.8875,51.9915,NL,EU
3159,35723,68.4085,43.3685,KZ,AS
3160,35724,127.0985,37.1995,KR,AS


In [14]:
df_merged.to_csv('../metadata/probe-coords-cc.csv', header=True, sep=';')

In [15]:
import folium
from folium.plugins import MarkerCluster

In [37]:
map = folium.Map([10,5],
                 height='80%', 
                 tiles="cartodbpositron", zoom_start=2, min_zoom=0, max_zoom=50)

marker_cluster = MarkerCluster().add_to(map)
feature_probes = folium.FeatureGroup(name='Probes')
feature_probes.add_to(map)

for i in range(len(df_merged)):
        lat = df.iloc[i]['lat']
        long = df.iloc[i]['long']
        radius=0

        if df_merged.iloc[i]['probe_id'] in local_dot:
            folium.Marker(location = [lat, long]).add_to(marker_cluster)
            folium.CircleMarker(location=[lat, long],
                    radius=2,
                    color='green', fill=False, weight=0.8).add_to(feature_probes)
        
        else:
            folium.CircleMarker(location = [lat, long], radius=radius, fill=False, weight=0).add_to(marker_cluster)
        
            folium.CircleMarker(location=[lat, long],
                                radius=2,
                                color='blue', fill=False, weight=0.5).add_to(feature_probes)


In [38]:
map

In [39]:
map.save('map.html')